In [1]:
from datetime import datetime
from operator import itemgetter
from os import path

from astropy.table import Table
import numpy as np
from skynet.api.v20 import errors
from skynet.api.v20.methods import download, groups, obs, scopes

In [15]:
# define parameters to control the code below

data_directory = "Data"
start_time = "2024-09-06T00:00:00" # only download files after this time
# start_time = "2024-08-15 00:00:00" # only download files after this time
observation_criteria = {"groupId": groups.get("Majewski Research Group")["id"]}
exposure_criteria = observation_criteria

# scope_id = scopes.query(include="*", **{"name": "RRRT"})[0]["id"]

observation_criteria

{'groupId': 816}

In [16]:
# retrieve list of observations happening after start_time and meeting observation_criteria
# write to csv just in case it's useful

observation_fields = {
    "id": "Observation ID",
    "state": "Observation state",
    "priority": "Priority",
    "name": "Target Name",
    "user.username": "Username",
    "group.name": "Group",
    "raHours": "RA",
    "decDegs": "Dec",
    "currentTelescope.name": "Telescope",
    "teleOwner.name": "Scope Owner",
    "timeIn": "Submitted",
}

observation_results = obs.query(include=",".join(observation_fields.keys()), after=start_time, **observation_criteria)

observations = Table()
for field, col_name in observation_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 1:
            observations[col_name] = list(map(itemgetter(field), observation_results))
        case 2:
            field_dicts = list(map(itemgetter(field_parts[0]), observation_results))
            observations[col_name] = [field_dict[field_parts[1]] if field_dict is not None else "" for field_dict in field_dicts]
observations.write("skynet_observations.csv", overwrite=True)

observations

Observation ID,Observation state,Priority,Target Name,Username,Group,RA,Dec,Telescope,Scope Owner,Submitted
int32,str9,int32,str13,str7,str23,float64,float64,str4,str3,str19
11636835,canceled,1,tic 278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,,UVA,2024-09-06T00:52:37
11636836,canceled,3,tic 278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,,UVA,2024-09-06T00:56:21
11636838,completed,4,tic278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,RRRT,UVA,2024-09-06T01:02:26
11636840,completed,1,tic 278465736,gnh3pgz,Majewski Research Group,21.65336111111111,51.34663888888889,RRRT,UVA,2024-09-06T01:06:47


In [17]:
# retrieve list of images happening after start_time and meeting exposure_criteria
# write to csv just in case it's useful

exposure_fields = {
    "exps.obsId": "Observation ID",
    "exps.expNum": "Exposure Num",
    "exps.imageId": "Image ID",
    "exps.obs.name": "Name",
    "exps.type": "Type",
    "exps.filterName": "Filter Name",
    "exps.timeTaken": "Time Taken",
    "exps.state": "Image State",
    "exps.expLengthUsed": "Exposure",
    "exps.binningUsed": "Binning",
    "exps.gain": "Gain",
    "exps.camera.name": "Camera",
}

exposure_results = obs.query(include=",".join(exposure_fields.keys()), after=start_time, **exposure_criteria)

exp_structs = [image for result in exposure_results for image in result["exps"]]
images = Table()
for field, col_name in exposure_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 2:
            images[col_name] = list(map(itemgetter(field_parts[1]), exp_structs))
        case 3:
            field_dicts = list(map(itemgetter(field_parts[1]), exp_structs))
            images[col_name] = [field_dict[field_parts[2]] if field_dict is not None else "" for field_dict in field_dicts]
images.write("skynet_images.csv", overwrite=True)

images

Observation ID,Exposure Num,Image ID,Name,Type,Filter Name,Time Taken,Image State,Exposure,Binning,Gain,Camera
int32,int32,str9,str13,str5,str1,object,str8,object,int32,object,str40
11636835,0,r43942062,tic 278465736,light,V,None,canceled,None,1,None,
11636835,1,r43942063,tic 278465736,light,V,None,canceled,None,1,None,
11636835,2,r43942064,tic 278465736,light,V,None,canceled,None,1,None,
11636836,0,r43942065,tic 278465736,light,V,None,canceled,None,1,None,
11636836,1,r43942066,tic 278465736,light,V,None,canceled,None,1,None,
11636836,2,r43942067,tic 278465736,light,V,None,canceled,None,1,None,
11636838,0,r43942069,tic278465736,light,V,2024-09-06T01:03:16,archived,10.0,2,1.31791,UVA SBIG STX-16803 with FLI Filter Wheel
11636838,1,r43942070,tic278465736,light,V,2024-09-06T01:03:34,archived,12.0,2,1.31791,UVA SBIG STX-16803 with FLI Filter Wheel
11636838,2,r43942071,tic278465736,light,V,2024-09-06T01:03:53,archived,15.0,2,1.31791,UVA SBIG STX-16803 with FLI Filter Wheel


In [31]:
# do the actual download of the identified files
# skip files that already exist

def download_file_part(data_directory, obs_id, target, num_parts, part_num):
    if num_parts > 1:
        filename = f"{data_directory}/{obs_id}-{target}_{part_num}.zip"
    else:
        filename = f"{data_directory}/{obs_id}-{target}.zip"
    if path.isfile(filename):
        print(f"  File {filename} already downloaded")
        return
    print(f"  Downloading part {part_num}...", end=" ")
    _, data = download.fits(**{"obs": obs_id}, total_parts=num_parts, part=part_num)
    filesize = len(data)
    duration = datetime.now() - start
    if filesize > 0:
        with open(filename, "wb") as f:
            f.write(data)
        print(f"got {filesize/1024/1024:7.1f}MB at {filesize / duration.seconds * 8 / 1024 / 1024:5.2f}Mbps saved to {filename}")

for observation in observations:
    obs_id = observation[observation_fields["id"]]
    target = observation[observation_fields["name"]].strip().replace(" ", "_")
    print(f"Downloading observation {obs_id:10d}")
    start = datetime.now()
    num_images = len(images[images["Observation ID"] == obs_id])
    num_parts = num_images // 1000 + 1
    for part_num in range(1, num_parts + 1):
        try:
            download_file_part(data_directory, obs_id, target, num_parts, part_num)
        except errors.BadRequestError:
            print(f"  skipped part {part_num}- got bad request")
            continue
print("---Downloads Complete---")

---Downloads Complete---


In [28]:
num_images = 1008
num_parts = num_images // 1000 + 1
list(range(1, num_parts+1))

[1, 2]